In [138]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")
import asyncio

In [139]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpio(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpio(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}


In [140]:
def Change_LED(new_input, element):
    global LED_Status, user_input
    reset_gpio()
    LED_Status[element] = new_input
    user_input = False
    #print(LED_Status)
    

In [141]:
def LED():
    global LED_Status, PWM_on, PWM_off, PWM_run_time, sleep_time
    if LED_Status[0] == 0: #IF OFF
        time.sleep(sleep_time)
    else: #IF ON
        if LED_Status[3] == 3: #full bright (NO PWM)
            if LED_Status[2] == 0: #If NO PWM & Cycle Color
                for i in range(1,4): #cycle write from 1 (blue), 2 (red) and 3 (green)
                    write_gpio(i,1) #turn on  LED
                    time.sleep(sleep_time) #sleep with ON
                    write_gpio(i,0) #turn off LED
                    if LED_Status[1] == 1: #if supposed to blink, sleep
                        time.sleep(sleep_time)
            else: #IF NO PWM & Solid Color
                write_gpio(LED_Status[2],1)
                time.sleep(sleep_time)
                if LED_Status[1] == 1: #if supposed to blink, sleep
                        write_gpio(LED_Status[2],0)
                        time.sleep(sleep_time)
        else: #If NOT full brightness (PWM needed)
            if LED_Status[2] == 0: #PWM & cycle thru colors
                for i in range(1,4):
                    for j in range(PWM_run_time):
                        write_gpio(i,1) #turn on LED
                        time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                        write_gpio(i,0) #turn off LED
                        time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                    if LED_Status[1] == 1: #If blinking
                        time.sleep(sleep_time)
            else:
                for i in range(PWM_run_time):
                    write_gpio(LED_Status[2],1) #turn on LED
                    time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                    write_gpio(LED_Status[2],0) #turn off LED
                    time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                if LED_Status[1] == 1: #If Blinking
                    time.sleep(sleep_time)

In [142]:
LED_Status = [0, 0, 2, 3] #default status
#LED_Status = [1, 0, 2, 3] #testing status
#          = [off/on, solid/blink, cycle/blue/red/green, brightness_level]

Duty_Cycle = [0.04736614270344993, 0.217637640824031, 0.5310492251033824, 1.0] #25%, 50%, 75%, and 100% brightness.
#time_on = duty_cycle * (1/frequency) && time_off = (1-duty_cycle) * (1/frequency)
frequency = 20 #for the PWM Brightness
PWM_on = []
for i in range(4):
    PWM_on.append(Duty_Cycle[i]/frequency)
PWM_off = []
for i in range(4):
    PWM_off.append((1-Duty_Cycle[i])/frequency)
sleep_time = 1
PWM_run_time = frequency*sleep_time

user_input = False

change_value = 0
change_element = 0

reset_gpio()

In [143]:
def testLED():
    global LED_State, test, change_value, change_element, user_input
    if test == 3:                 #1) TURN ON
        change_value = 1
        change_element = 0
        #LED_Status[0] = 1
        user_input = True
    elif test == 5:               #2) Blue
        change_value = 1
        change_element = 2
        #LED_Status[2] = 3
        user_input = True
    elif test == 7:               #3) Green
        change_value = 3
        change_element = 2
        #LED_Status[2] = 1
        user_input = True
    elif test == 9:               #4) Cycle thru
        change_value = 0
        change_element = 2
        #LED_Status[2] = 0
        user_input = True
    elif test == 11:              #5) Blink (while cycling)
        change_value = 1
        change_element = 1
        #LED_Status[1] = 1
        user_input = True
    elif test == 13:              #6) Red (While Blinking)
        change_value = 2
        change_element = 2
        #LED_Status[2] = 2
        user_input = True
    elif test == 15:              #7) Brightness LOW (Red & blinking)
        change_value = 0
        change_element = 3
        LED_Status[3] = 0
        user_input = True
    elif test == 17:              #8) Cycle Colors (While low brightness & Blinking)
        change_value = 0
        change_element = 2
        LED_Status[2] = 0
        user_input = True
    elif test == 19:              #9) Red (While low brightness & Blinking)
        change_value = 2
        change_element = 2
        #LED_Status[2] = 2
        user_input = True
    elif test == 21:              #10) Solid (While Red and low brightness)
        change_value = 0
        change_element = 1
        #LED_Status[1] = 0
        user_input = True
    elif test == 23:              #11) brightness up 1 (While Red and solid)
        change_value = 1
        change_element = 3
        #LED_Status[1] = 0
        user_input = True
    elif test == 25:              #12) brightness up 1 (While Red and solid)
        change_value = 2
        change_element = 3
        #LED_Status[1] = 0
        user_input = True
    elif test == 27:              #13) brightness up 1 - full bright (While Red and solid)
        change_value = 3
        change_element = 3
        #LED_Status[1] = 0
        user_input = True

In [144]:
reset_gpio()

while True:
    LED()
    #time.sleep(sleep_time)
    #user_input(1)
    if user_input == True:
        Change_LED(change_value, change_element)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


KeyboardInterrupt: 

In [117]:
reset_gpio()
test = 0
LED_Status = [0, 0, 2, 3]

while True:
    print(test)
    LED()
    test = test+1
    if test >= 3:
        #print('LED_State will be changed to:')
        testLED()
    #time.sleep(sleep_time)
    #user_input(1)
    if user_input == True:
        Change_LED(change_value, change_element)